# 🤖 RL Algorithm Comparison: DQN vs PPO vs A2C
## Enhanced NeuroFire Project Analysis

**Objective**: Compare three state-of-the-art Deep Reinforcement Learning algorithms for autonomous firefighter drone control, with comprehensive performance analysis and visualizations.

---

## 📋 Table of Contents
1. **Environment Setup & Dependencies**
2. **Custom Environment Implementation** 
3. **DQN Agent Architecture & Training**
4. **PPO Agent Architecture & Training**
5. **A2C Agent Architecture & Training**
6. **Training Orchestration & Monitoring**
7. **Performance Evaluation & Metrics**
8. **Comparative Visualization & Analysis**
9. **Interactive Simulations**
10. **Final Results & Recommendations**

---

## 🎯 Project Goals
- Train and compare 3 RL algorithms on NeuroFire environment
- Visualize learning curves and convergence behavior
- Analyze strengths/weaknesses of each approach
- Generate actionable recommendations for algorithm selection

In [25]:
# SECTION 1: Environment Setup & Dependencies
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque, namedtuple
import random
import time

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {DEVICE}")

# Set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE.type == "cuda":
    torch.cuda.manual_seed(SEED)

# Plotting styles
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

print("✅ All dependencies imported successfully!")

✅ Using device: cpu
✅ All dependencies imported successfully!


# SECTION 2: Custom NeuroFire Environment

A simplified fire-fighting environment where an AI agent learns to navigate and extinguish fires in a grid-based forest.

**State**: Direction to nearest fire, obstacles around agent (11 inputs)
**Actions**: Move straight, turn right, turn left (3 discrete actions)  
**Reward**: +10 for extinguishing fire, -10 for hitting obstacle, -1 per step
**Goal**: Maximize total reward by efficiently navigating and extinguishing fires

In [26]:
# Simplified NeuroFire Environment for Algorithm Comparison
class NeuroFireSimplified:
    """Simplified Fire Environment for RL Algorithm Comparison"""
    
    def __init__(self, grid_size=20, fire_density=0.1):
        self.grid_size = grid_size
        self.fire_density = fire_density
        self.state_size = 11  # Sensor inputs
        self.action_size = 3  # Straight, Right, Left
        self.reset()
    
    def reset(self):
        """Reset environment to initial state"""
        # Agent position
        self.agent_x = self.grid_size // 2
        self.agent_y = self.grid_size // 2
        self.agent_direction = 0  # 0=Up, 1=Right, 2=Down, 3=Left
        
        # Fire positions (random)
        self.fires = set()
        for _ in range(int(self.grid_size**2 * self.fire_density)):
            self.fires.add((np.random.randint(0, self.grid_size),
                          np.random.randint(0, self.grid_size)))
        
        # Obstacles (walls at boundaries + some internal)
        self.obstacles = set()
        for i in range(self.grid_size):
            self.obstacles.add((i, 0))
            self.obstacles.add((i, self.grid_size-1))
            self.obstacles.add((0, i))
            self.obstacles.add((self.grid_size-1, i))
        
        self.steps = 0
        self.max_steps = 200
        
        return self._get_state()
    
    def _get_state(self):
        """Get sensor state (11-dimensional observation)"""
        state = np.zeros(self.state_size, dtype=np.float32)
        
        # Direction encoding
        state[0] = self.agent_direction / 4.0
        
        # Obstacle sensors (8 directions)
        directions = [(-1, 0), (-1, 1), (0, 1), (1, 1),
                     (1, 0), (1, -1), (0, -1), (-1, -1)]
        for i, (dx, dy) in enumerate(directions):
            nx, ny = self.agent_x + dx, self.agent_y + dy
            state[i+1] = 1.0 if (nx, ny) in self.obstacles else 0.0
        
        # Fire detection (nearest fire direction)
        if self.fires:
            fire_distances = [(((self.agent_x - fx)**2 + (self.agent_y - fy)**2)**0.5, fx, fy)
                            for fx, fy in self.fires]
            _, nearest_fx, nearest_fy = min(fire_distances)
            
            # Fire bearing (-1 to 1)
            fire_dy = nearest_fy - self.agent_y
            fire_dx = nearest_fx - self.agent_x
            state[10] = np.clip(fire_dx / (self.grid_size / 2), -1, 1)
        
        return state
    
    def step(self, action):
        """Execute action: 0=Straight, 1=Right, 2=Left"""
        self.steps += 1
        reward = 0
        
        # Update direction and position
        if action == 1:  # Turn right
            self.agent_direction = (self.agent_direction + 1) % 4
        elif action == 2:  # Turn left
            self.agent_direction = (self.agent_direction - 1) % 4
        
        # Move forward
        dx = [0, 1, 0, -1][self.agent_direction]
        dy = [-1, 0, 1, 0][self.agent_direction]
        
        new_x = self.agent_x + dx
        new_y = self.agent_y + dy
        
        # Check collision with obstacles
        if (new_x, new_y) in self.obstacles:
            reward -= 10  # Penalty for hitting wall
        else:
            self.agent_x = new_x
            self.agent_y = new_y
        
        # Check if fire extinguished
        if (self.agent_x, self.agent_y) in self.fires:
            self.fires.remove((self.agent_x, self.agent_y))
            reward += 10  # Reward for extinguishing fire
        
        # Step penalty
        reward -= 0.01
        
        done = self.steps >= self.max_steps or len(self.fires) == 0
        
        return self._get_state(), reward, done
    
    def render(self):
        """Simple text rendering"""
        print("Grid (Agent: @, Fire: F, Wall: #):")
        for y in range(self.grid_size):
            row = ""
            for x in range(self.grid_size):
                if (x, y) == (self.agent_x, self.agent_y):
                    row += "@"
                elif (x, y) in self.fires:
                    row += "F"
                elif (x, y) in self.obstacles:
                    row += "#"
                else:
                    row += "."
            print(row)

# Test environment
print("Testing NeuroFireSimplified environment...")
test_env = NeuroFireSimplified()
state = test_env.reset()
print(f"✅ Initial state shape: {state.shape}")
print(f"✅ State sample: {state[:5]}")

for i in range(5):
    action = np.random.randint(0, 3)
    state, reward, done = test_env.step(action)
    print(f"   Step {i+1}: action={action}, reward={reward:.2f}, done={done}")

Testing NeuroFireSimplified environment...
✅ Initial state shape: (11,)
✅ State sample: [0. 0. 0. 0. 0.]
   Step 1: action=0, reward=-0.01, done=False
   Step 2: action=1, reward=-0.01, done=False
   Step 3: action=0, reward=-0.01, done=False
   Step 4: action=1, reward=-0.01, done=False
   Step 5: action=1, reward=-0.01, done=False


# SECTION 3: DQN Agent (Deep Q-Network)

**Key Characteristics:**
- **Learning Type**: Off-policy (learns from exploration data)
- **Architecture**: Value-based (estimates Q-values)
- **Experience Replay**: Stores and samples past transitions
- **Target Network**: Separate network for stable Q-target
- **Epsilon-Greedy**: Exploration-exploitation trade-off

**Advantages:**
✓ Sample efficient (off-policy learning)  
✓ Stable training with target network  
✓ Works with discrete actions  

**Disadvantages:**
✗ Can overestimate Q-values  
✗ Requires careful hyperparameter tuning

In [27]:
# DQN Implementation
class DQNNetwork(nn.Module):
    """Deep Q-Network"""
    def __init__(self, state_size, action_size, hidden_size=128):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(state_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, action_size)
        )
    
    def forward(self, x):
        return self.network(x)


class ReplayBuffer:
    """Experience Replay Buffer"""
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)
        self.Transition = namedtuple('Transition', 
                                    ('state', 'action', 'reward', 'next_state', 'done'))
    
    def push(self, *args):
        self.buffer.append(self.Transition(*args))
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        return self.Transition(*zip(*batch))
    
    def __len__(self):
        return len(self.buffer)


class DQNAgent:
    """Deep Q-Network Agent"""
    def __init__(self, state_size, action_size, lr=1e-4, gamma=0.99):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        
        # Networks
        self.policy_net = DQNNetwork(state_size, action_size).to(DEVICE)
        self.target_net = DQNNetwork(state_size, action_size).to(DEVICE)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        
        # Optimizer and memory
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.memory = ReplayBuffer()
        
        # Metrics
        self.losses = []
        self.rewards = []
        self.q_values = []
    
    def select_action(self, state, training=True):
        """Epsilon-greedy action selection"""
        if training and np.random.random() < self.epsilon:
            return np.random.randint(0, self.action_size)
        
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0).to(DEVICE)
            q_values = self.policy_net(state_t)
            return q_values.argmax(1).item()
    
    def remember(self, state, action, reward, next_state, done):
        """Store transition in memory"""
        self.memory.push(state, action, reward, next_state, done)
    
    def train_step(self, batch_size=64):
        """Training step"""
        if len(self.memory) < batch_size:
            return 0
        
        batch = self.memory.sample(batch_size)
        
        states = torch.FloatTensor(np.array(batch.state)).to(DEVICE)
        actions = torch.LongTensor(np.array(batch.action)).unsqueeze(1).to(DEVICE)
        rewards = torch.FloatTensor(np.array(batch.reward)).to(DEVICE)
        next_states = torch.FloatTensor(np.array(batch.next_state)).to(DEVICE)
        dones = torch.FloatTensor(np.array(batch.done)).to(DEVICE)
        
        # Current Q values
        q_values = self.policy_net(states).gather(1, actions).squeeze()
        
        # Target Q values
        with torch.no_grad():
            next_q_values = self.target_net(next_states).max(1)[0]
            target_q_values = rewards + (1 - dones) * self.gamma * next_q_values
        
        # Loss
        loss = F.mse_loss(q_values, target_q_values)
        
        # Backprop
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()
        
        # Update target network periodically
        if len(self.memory) % 500 == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
        
        # Decay epsilon
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        
        self.losses.append(loss.item())
        return loss.item()

print("✅ DQN Agent implemented")

✅ DQN Agent implemented


# SECTION 4: PPO Agent (Proximal Policy Optimization)

**Key Characteristics:**
- **Learning Type**: On-policy (learns from recent data)
- **Architecture**: Actor-Critic (policy + value)
- **Clipped Objective**: Prevents extreme policy updates
- **GAE**: Generalized Advantage Estimation for variance reduction
- **Multiple Epochs**: Updates on same batch multiple times

**Advantages:**
✓ More stable than policy gradient methods  
✓ Better sample efficiency than vanilla A3C  
✓ Easy to parallelize  

**Disadvantages:**
✗ More complex hyperparameters  
✗ Higher computational cost

In [28]:
# PPO Implementation
class ActorCritic(nn.Module):
    """Actor-Critic network for PPO"""
    def __init__(self, state_size, action_size, hidden_size=128):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(state_size, hidden_size),
            nn.ReLU()
        )
        self.actor = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, action_size),
            nn.Softmax(dim=-1)
        )
        self.critic = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
    
    def forward(self, x):
        shared = self.shared(x)
        return self.actor(shared), self.critic(shared)


class PPOAgent:
    """Proximal Policy Optimization Agent"""
    def __init__(self, state_size, action_size, lr=3e-4, gamma=0.99, 
                 gae_lambda=0.95, clip_ratio=0.2, epochs=4):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.clip_ratio = clip_ratio
        self.epochs = epochs
        
        self.policy = ActorCritic(state_size, action_size).to(DEVICE)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        
        # Storage
        self.states = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.log_probs = []
        
        # Metrics
        self.losses = []
    
    def select_action(self, state, training=True):
        """Select action and compute log probability"""
        state_t = torch.FloatTensor(state).unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            probs, value = self.policy(state_t)
        
        if training:
            dist = Categorical(probs)
            action = dist.sample()
            log_prob = dist.log_prob(action)
            
            self.states.append(state)
            self.actions.append(action.item())
            self.values.append(value.item())
            self.log_probs.append(log_prob)
            
            return action.item()
        else:
            return probs.argmax(1).item()
    
    def store_reward(self, reward):
        """Store reward"""
        self.rewards.append(reward)
    
    def compute_advantages(self, next_value=0):
        """Compute GAE advantages"""
        advantages = []
        gae = 0
        
        for t in reversed(range(len(self.rewards))):
            if t == len(self.rewards) - 1:
                next_val = next_value
            else:
                next_val = self.values[t + 1]
            
            delta = self.rewards[t] + self.gamma * next_val - self.values[t]
            gae = delta + self.gamma * self.gae_lambda * gae
            advantages.insert(0, gae)
        
        return torch.FloatTensor(advantages).to(DEVICE)
    
    def train(self, next_state, done):
        """Update policy using PPO"""
        if len(self.states) == 0:
            return 0
        
        # Compute next value for bootstrapping
        if done:
            next_value = 0
        else:
            with torch.no_grad():
                _, v = self.policy(torch.FloatTensor(next_state).unsqueeze(0).to(DEVICE))
                next_value = v.item()
        
        # Compute advantages and returns
        advantages = self.compute_advantages(next_value)
        returns = advantages + torch.FloatTensor(self.values).to(DEVICE)
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # Convert to tensors
        states = torch.FloatTensor(np.array(self.states)).to(DEVICE)
        actions = torch.LongTensor(np.array([self.actions])).squeeze().to(DEVICE)
        old_log_probs = torch.stack(self.log_probs).to(DEVICE)
        
        # PPO update
        total_loss = 0
        for _ in range(self.epochs):
            probs, values = self.policy(states)
            dist = Categorical(probs)
            new_log_probs = dist.log_prob(actions)
            entropy = dist.entropy().mean()
            
            # Policy loss
            ratios = torch.exp(new_log_probs - old_log_probs)
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.clip_ratio, 1 + self.clip_ratio) * advantages
            policy_loss = -torch.min(surr1, surr2).mean()
            
            # Value loss
            value_loss = F.mse_loss(values.squeeze(), returns)
            
            # Total loss
            loss = policy_loss + 0.5 * value_loss - 0.01 * entropy
            
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 0.5)
            self.optimizer.step()
            
            total_loss += loss.item()
        
        self.losses.append(total_loss / self.epochs)
        
        # Clear storage
        self.states, self.actions, self.rewards = [], [], []
        self.values, self.log_probs = [], []
        
        return total_loss / self.epochs

print("✅ PPO Agent implemented")

✅ PPO Agent implemented


# SECTION 5: A2C Agent (Advantage Actor-Critic)

**Key Characteristics:**
- **Learning Type**: On-policy (learns from recent data)
- **Architecture**: Actor-Critic (policy + value)
- **Synchronous**: No parallel workers (unlike A3C)
- **TD Error**: Uses temporal difference for advantage
- **Entropy Bonus**: Encourages exploration

**Advantages:**
✓ Simple to implement  
✓ Faster convergence than vanilla PG  
✓ Lower variance than reinforce  

**Disadvantages:**
✗ Less stable than PPO  
✗ Sensitive to hyperparameters

In [29]:
# A2C Implementation
class A2CAgent:
    """Advantage Actor-Critic Agent"""
    def __init__(self, state_size, action_size, lr=3e-4, gamma=0.99,
                 value_coef=0.5, entropy_coef=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        
        self.policy = ActorCritic(state_size, action_size).to(DEVICE)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        
        # Metrics
        self.losses = []
    
    def select_action(self, state, training=True):
        """Select action using policy"""
        state_t = torch.FloatTensor(state).unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            probs, value = self.policy(state_t)
        
        if training:
            dist = Categorical(probs)
            action = dist.sample()
            log_prob = dist.log_prob(action)
            return action.item(), log_prob, value
        else:
            return probs.argmax(1).item(), None, None
    
    def train(self, state, action, reward, next_state, done, log_prob):
        """A2C update"""
        state_t = torch.FloatTensor(state).unsqueeze(0).to(DEVICE)
        next_state_t = torch.FloatTensor(next_state).unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            _, v_current = self.policy(state_t)
            _, v_next = self.policy(next_state_t)
        
        # TD target
        if done:
            td_target = reward
        else:
            td_target = reward + self.gamma * v_next.item()
        
        # Get fresh values and probs for loss computation
        probs, v_pred = self.policy(state_t)
        dist = Categorical(probs)
        
        # Advantage
        advantage = td_target - v_pred.item()
        
        # Losses
        actor_loss = -log_prob * advantage
        critic_loss = (v_pred.squeeze() - td_target) ** 2
        entropy = dist.entropy()
        
        # Total loss
        loss = actor_loss + self.value_coef * critic_loss - self.entropy_coef * entropy
        
        # Update
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 0.5)
        self.optimizer.step()
        
        self.losses.append(loss.item())
        return loss.item()

print("✅ A2C Agent implemented")

✅ A2C Agent implemented


# SECTION 6: Training Orchestration

Now we'll train all three agents on the NeuroFire environment and monitor their progress.

In [ ]:
# Training function
def train_agent(agent, agent_name, episodes=200):
    """Train an agent on the NeuroFire environment"""
    env = NeuroFireSimplified()
    
    episode_rewards = []
    mean_rewards = []
    best_reward = float('-inf')
    
    print(f"\n{'='*60}")
    print(f"🎯 Training {agent_name} Agent")
    print(f"{'='*60}")
    
    for episode in range(episodes):
        state = env.reset()
        episode_reward = 0
        done = False
        step = 0
        
        while not done and step < env.max_steps:
            # Select action
            if agent_name == "DQN":
                action = agent.select_action(state, training=True)
                next_state, reward, done = env.step(action)
                agent.remember(state, action, reward, next_state, done)
                agent.train_step()
                
            elif agent_name == "PPO":
                action = agent.select_action(state, training=True)
                next_state, reward, done = env.step(action)
                agent.store_reward(reward)
                
            elif agent_name == "A2C":
                action, log_prob, value = agent.select_action(state, training=True)
                next_state, reward, done = env.step(action)
                agent.train(state, action, reward, next_state, done, log_prob)
            
            episode_reward += reward
            state = next_state
            step += 1
        
        # PPO training at end of episode
        if agent_name == "PPO":
            agent.train(state, done)
        
        # Store metrics
        episode_rewards.append(episode_reward)
        mean_reward = np.mean(episode_rewards[-100:])
        mean_rewards.append(mean_reward)
        
        if episode_reward > best_reward:
            best_reward = episode_reward
        
        # Progress print
        if (episode + 1) % 50 == 0:
            print(f"Episode {episode+1:3d} | Reward: {episode_reward:7.2f} | "
                  f"Mean(100): {mean_reward:7.2f} | Best: {best_reward:7.2f}")
    
    print(f"{'='*60}")
    print(f"✅ Training Complete!\n")
    
    return {
        'rewards': episode_rewards,
        'mean_rewards': mean_rewards,
        'losses': agent.losses,
        'best_reward': best_reward
    }

# Train all three agents
print("\\n" + "🚀 "*30)
print("STARTING AGENT TRAINING")
print("🚀 "*30)

# Environment info
env = NeuroFireSimplified()
print(f"\n📊 Environment Info:")
print(f"   State Size: {env.state_size}")
print(f"   Action Size: {env.action_size}")
print(f"   Max Steps per Episode: {env.max_steps}")

# Initialize agents
print(f"\n🤖 Initializing Agents...")
dqn_agent = DQNAgent(env.state_size, env.action_size)
ppo_agent = PPOAgent(env.state_size, env.action_size)
a2c_agent = A2CAgent(env.state_size, env.action_size)
print(f"✅ All agents initialized")

# Train agents
num_episodes = 200
dqn_results = train_agent(dqn_agent, "DQN", episodes=num_episodes)
ppo_results = train_agent(ppo_agent, "PPO", episodes=num_episodes)
a2c_results = train_agent(a2c_agent, "A2C", episodes=num_episodes)

\n🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 
STARTING AGENT TRAINING
🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 

📊 Environment Info:
   State Size: 11
   Action Size: 3
   Max Steps per Episode: 200

🤖 Initializing Agents...
✅ All agents initialized

🎯 Training DQN Agent
Episode  50 | Reward:   -2.00 | Mean(100):   57.60 | Best:  168.00
Episode 100 | Reward:   48.00 | Mean(100):   65.90 | Best:  218.00
Episode 150 | Reward:    8.00 | Mean(100):   84.80 | Best:  238.00


# SECTION 7: Performance Evaluation & Metrics

Evaluate the trained agents on the NeuroFire environment without exploration.

In [ ]:
# Evaluation function
def evaluate_agent(agent, agent_name, num_eval_episodes=20):
    """Evaluate agent performance without training"""
    env = NeuroFireSimplified()
    
    eval_rewards = []
    eval_fires_extinguished = []
    
    for _ in range(num_eval_episodes):
        state = env.reset()
        episode_reward = 0
        fires_start = len(env.fires)
        done = False
        
        while not done:
            if agent_name == "DQN":
                action = agent.select_action(state, training=False)
            elif agent_name == "PPO":
                action = agent.select_action(state, training=False)
            elif agent_name == "A2C":
                action, _, _ = agent.select_action(state, training=False)
            
            next_state, reward, done = env.step(action)
            episode_reward += reward
            state = next_state
        
        fires_extinguished = fires_start - len(env.fires)
        eval_rewards.append(episode_reward)
        eval_fires_extinguished.append(fires_extinguished)
    
    return {
        'rewards': eval_rewards,
        'fires_extinguished': eval_fires_extinguished,
        'mean_reward': np.mean(eval_rewards),
        'std_reward': np.std(eval_rewards),
        'mean_fires': np.mean(eval_fires_extinguished)
    }

# Evaluate all agents
print("\n" + "="*60)
print("📊 EVALUATING AGENTS (NO TRAINING)")
print("="*60)

dqn_eval = evaluate_agent(dqn_agent, "DQN", num_eval_episodes=20)
ppo_eval = evaluate_agent(ppo_agent, "PPO", num_eval_episodes=20)
a2c_eval = evaluate_agent(a2c_agent, "A2C", num_eval_episodes=20)

print("\n✅ Evaluation Complete\n")

# Print evaluation results
print(f"{'Agent':<10} {'Mean Reward':>15} {'Std Dev':>15} {'Avg Fires':>15}")
print("-"*55)
print(f"{'DQN':<10} {dqn_eval['mean_reward']:>15.2f} {dqn_eval['std_reward']:>15.2f} {dqn_eval['mean_fires']:>15.2f}")
print(f"{'PPO':<10} {ppo_eval['mean_reward']:>15.2f} {ppo_eval['std_reward']:>15.2f} {ppo_eval['mean_fires']:>15.2f}")
print(f"{'A2C':<10} {a2c_eval['mean_reward']:>15.2f} {a2c_eval['std_reward']:>15.2f} {a2c_eval['mean_fires']:>15.2f}")
print("-"*55)

# SECTION 8: Comprehensive Visualization & Analysis

In [ ]:
# Create comprehensive comparison visualizations
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

colors = {'DQN': '#2E86AB', 'PPO': '#A23B72', 'A2C': '#F18F01'}

# 1. Learning Curves (Smoothed)
ax1 = fig.add_subplot(gs[0, :2])
window = 20
for name, results in [('DQN', dqn_results), ('PPO', ppo_results), ('A2C', a2c_results)]:
    smoothed = np.convolve(results['mean_rewards'], np.ones(window)/window, mode='valid')
    ax1.plot(smoothed, label=name, linewidth=2.5, color=colors[name])
ax1.set_xlabel('Episode', fontsize=11, fontweight='bold')
ax1.set_ylabel('Mean Reward (100-ep window)', fontsize=11, fontweight='bold')
ax1.set_title('🎯 Learning Curves Comparison', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10, loc='lower right')
ax1.grid(True, alpha=0.3)

# 2. Best Reward Achieved
ax2 = fig.add_subplot(gs[0, 2])
names = ['DQN', 'PPO', 'A2C']
best_rewards = [dqn_results['best_reward'], ppo_results['best_reward'], a2c_results['best_reward']]
bars = ax2.bar(names, best_rewards, color=[colors[n] for n in names], alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Best Reward', fontsize=11, fontweight='bold')
ax2.set_title('🏆 Best Rewards', fontsize=13, fontweight='bold')
for bar, val in zip(bars, best_rewards):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# 3. Training Loss
ax3 = fig.add_subplot(gs[1, 0])
for name, results in [('DQN', dqn_results), ('PPO', ppo_results), ('A2C', a2c_results)]:
    if results['losses']:
        ax3.plot(results['losses'][:300], label=name, linewidth=1.5, alpha=0.8, color=colors[name])
ax3.set_xlabel('Training Step', fontsize=11, fontweight='bold')
ax3.set_ylabel('Loss', fontsize=11, fontweight='bold')
ax3.set_title('📉 Training Loss', fontsize=13, fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# 4. Evaluation Rewards (Boxplot)
ax4 = fig.add_subplot(gs[1, 1])
eval_data = [dqn_eval['rewards'], ppo_eval['rewards'], a2c_eval['rewards']]
bp = ax4.boxplot(eval_data, labels=['DQN', 'PPO', 'A2C'], patch_artist=True)
for patch, name in zip(bp['boxes'], names):
    patch.set_facecolor(colors[name])
    patch.set_alpha(0.7)
ax4.set_ylabel('Episode Reward', fontsize=11, fontweight='bold')
ax4.set_title('📦 Reward Distribution', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

# 5. Mean Evaluation Performance
ax5 = fig.add_subplot(gs[1, 2])
mean_eval = [dqn_eval['mean_reward'], ppo_eval['mean_reward'], a2c_eval['mean_reward']]
std_eval = [dqn_eval['std_reward'], ppo_eval['std_reward'], a2c_eval['std_reward']]
bars = ax5.bar(names, mean_eval, yerr=std_eval, color=[colors[n] for n in names], 
              alpha=0.7, edgecolor='black', linewidth=1.5, capsize=5)
ax5.set_ylabel('Mean Reward', fontsize=11, fontweight='bold')
ax5.set_title('⭐ Evaluation Performance', fontsize=13, fontweight='bold')
for bar, val in zip(bars, mean_eval):
    height = bar.get_height()
    ax5.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# 6. Fires Extinguished
ax6 = fig.add_subplot(gs[2, 0])
fires = [dqn_eval['mean_fires'], ppo_eval['mean_fires'], a2c_eval['mean_fires']]
bars = ax6.bar(names, fires, color=[colors[n] for n in names], alpha=0.7, edgecolor='black', linewidth=1.5)
ax6.set_ylabel('Avg Fires Extinguished', fontsize=11, fontweight='bold')
ax6.set_title('🔥 Fire Suppression', fontsize=13, fontweight='bold')
for bar, val in zip(bars, fires):
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# 7. Convergence Speed (episodes to decent performance)
ax7 = fig.add_subplot(gs[2, 1])
convergence_threshold = 0.5 * max([dqn_results['best_reward'], ppo_results['best_reward'], a2c_results['best_reward']])

conv_episodes = []
for results in [dqn_results, ppo_results, a2c_results]:
    for i, reward in enumerate(results['mean_rewards']):
        if reward >= convergence_threshold:
            conv_episodes.append(i)
            break
    else:
        conv_episodes.append(len(results['mean_rewards']))

bars = ax7.bar(names, conv_episodes, color=[colors[n] for n in names], alpha=0.7, edgecolor='black', linewidth=1.5)
ax7.set_ylabel('Episodes', fontsize=11, fontweight='bold')
ax7.set_title('⚡ Convergence Speed', fontsize=13, fontweight='bold')
for bar, val in zip(bars, conv_episodes):
    height = bar.get_height()
    ax7.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(val)}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# 8. Stability (Coefficient of Variation)
ax8 = fig.add_subplot(gs[2, 2])
stability = [
    (dqn_eval['std_reward'] / (abs(dqn_eval['mean_reward']) + 1e-6)),
    (ppo_eval['std_reward'] / (abs(ppo_eval['mean_reward']) + 1e-6)),
    (a2c_eval['std_reward'] / (abs(a2c_eval['mean_reward']) + 1e-6))
]
bars = ax8.bar(names, stability, color=[colors[n] for n in names], alpha=0.7, edgecolor='black', linewidth=1.5)
ax8.set_ylabel('Coefficient of Variation', fontsize=11, fontweight='bold')
ax8.set_title('📊 Stability (lower=better)', fontsize=13, fontweight='bold')
for bar, val in zip(bars, stability):
    height = bar.get_height()
    ax8.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.suptitle('🤖 RL Algorithm Comparison: DQN vs PPO vs A2C on NeuroFire', 
            fontsize=16, fontweight='bold', y=0.995)

plt.savefig('neurofire_rl_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Comparison visualizations created!")

# SECTION 9: Algorithm Analysis Summary

In [ ]:
# Detailed Analysis and Comparison
print("\n" + "="*80)
print("🔬 COMPREHENSIVE ALGORITHM ANALYSIS".center(80))
print("="*80)

# Create detailed comparison table
comparison_data = {
    'Metric': [
        'Final Mean Reward',
        'Best Episode Reward',
        'Reward Std Dev',
        'Mean Fires Extinguished',
        'Training Stability (CV)',
        'Avg Fires per Episode',
        'Algorithm Type',
        'Sample Efficiency',
        'Implementation Complexity'
    ],
    'DQN': [
        f"{dqn_eval['mean_reward']:.2f}",
        f"{dqn_results['best_reward']:.2f}",
        f"{dqn_eval['std_reward']:.2f}",
        f"{dqn_eval['mean_fires']:.2f}",
        f"{(dqn_eval['std_reward'] / (abs(dqn_eval['mean_reward']) + 1e-6)):.3f}",
        f"{dqn_eval['mean_fires']:.2f}",
        "Value-Based",
        "Off-Policy ⭐⭐⭐",
        "Medium"
    ],
    'PPO': [
        f"{ppo_eval['mean_reward']:.2f}",
        f"{ppo_results['best_reward']:.2f}",
        f"{ppo_eval['std_reward']:.2f}",
        f"{ppo_eval['mean_fires']:.2f}",
        f"{(ppo_eval['std_reward'] / (abs(ppo_eval['mean_reward']) + 1e-6)):.3f}",
        f"{ppo_eval['mean_fires']:.2f}",
        "Policy-Based",
        "On-Policy ⭐⭐",
        "High"
    ],
    'A2C': [
        f"{a2c_eval['mean_reward']:.2f}",
        f"{a2c_results['best_reward']:.2f}",
        f"{a2c_eval['std_reward']:.2f}",
        f"{a2c_eval['mean_fires']:.2f}",
        f"{(a2c_eval['std_reward'] / (abs(a2c_eval['mean_reward']) + 1e-6)):.3f}",
        f"{a2c_eval['mean_fires']:.2f}",
        "Policy-Based",
        "On-Policy ⭐",
        "Low"
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n" + df_comparison.to_string(index=False))

# Key findings
print("\n" + "-"*80)
print("📌 KEY FINDINGS".center(80))
print("-"*80)

# Determine best in each category
best_final = max([
    ('DQN', dqn_eval['mean_reward']),
    ('PPO', ppo_eval['mean_reward']),
    ('A2C', a2c_eval['mean_reward'])
], key=lambda x: x[1])

best_stability = min([
    ('DQN', dqn_eval['std_reward']),
    ('PPO', ppo_eval['std_reward']),
    ('A2C', a2c_eval['std_reward'])
], key=lambda x: x[1])

best_fires = max([
    ('DQN', dqn_eval['mean_fires']),
    ('PPO', ppo_eval['mean_fires']),
    ('A2C', a2c_eval['mean_fires'])
], key=lambda x: x[1])

print(f"\n✨ Best Final Performance: {best_final[0]} ({best_final[1]:.2f})")
print(f"🎯 Most Stable: {best_stability[0]} (σ={best_stability[1]:.2f})")
print(f"🔥 Best at Fire Suppression: {best_fires[0]} ({best_fires[1]:.2f} fires)")

print("\n" + "-"*80)
print("💡 STRENGTHS & WEAKNESSES".center(80))
print("-"*80)

strengths_weaknesses = {
    'DQN': {
        'strengths': [
            '✓ Excellent sample efficiency (off-policy)',
            '✓ Stable with target network',
            '✓ Good for discrete action spaces',
            '✓ Experience replay reduces correlation'
        ],
        'weaknesses': [
            '✗ Can overestimate Q-values',
            '✗ Requires careful hyperparameter tuning',
            '✗ Memory overhead from replay buffer',
            '✗ Slower wall-clock training time'
        ]
    },
    'PPO': {
        'strengths': [
            '✓ Stable policy updates (clipped)',
            '✓ Better than REINFORCE',
            '✓ Easy to parallelize',
            '✓ Good balance of performance & complexity'
        ],
        'weaknesses': [
            '✗ On-policy: lower sample efficiency',
            '✗ Complex hyperparameter tuning',
            '✗ Requires more wall-clock time',
            '✗ GAE computation overhead'
        ]
    },
    'A2C': {
        'strengths': [
            '✓ Simple to implement',
            '✓ Fast convergence in some cases',
            '✓ Lower computational overhead',
            '✓ Good exploration with entropy bonus'
        ],
        'weaknesses': [
            '✗ Can be unstable',
            '✗ Sensitive to learning rate',
            '✗ High variance in gradient estimates',
            '✗ Slower convergence than DQN'
        ]
    }
}

for algo, info in strengths_weaknesses.items():
    print(f"\n{algo}:")
    print("  Strengths:")
    for s in info['strengths']:
        print(f"    {s}")
    print("  Weaknesses:")
    for w in info['weaknesses']:
        print(f"    {w}")

print("\n" + "="*80)

# SECTION 10: Recommendations & Conclusions

In [ ]:
print("\n" + "="*80)
print("🎯 ALGORITHM SELECTION GUIDE".center(80))
print("="*80)

guide = """
Choose DQN when:
  📊 You have limited computational resources (no parallelization needed)
  🎮 Working with discrete action spaces
  💾 You can afford memory for a large replay buffer
  ⚡ Sample efficiency is critical (limited environment interactions)
  
  BEST FOR: Limited interactions, discrete actions, stable learning

Choose PPO when:
  🚀 You can parallelize across multiple environments
  🎪 You want the best balance of performance and stability
  🔧 You prefer simpler tuning compared to DQN
  🏆 You need state-of-the-art performance on challenging tasks
  
  BEST FOR: Production systems, multi-environment training, robustness

Choose A2C when:
  🏃 You need fast prototyping and implementation
  💻 You have limited memory availability
  📚 You're learning RL concepts
  ⏱️ Training speed is more important than sample efficiency
  
  BEST FOR: Educational purposes, rapid prototyping, simple tasks

For NeuroFire Project:
  🔥 The autonomous firefighter drone environment benefits most from:
     1st Choice: PPO (best balance, robust to hyperparameters)
     2nd Choice: DQN (if training data is scarce)
     3rd Choice: A2C (for quick iterations during development)
"""

print(guide)

# Final recommendation
print("\n" + "="*80)
print("🏆 FINAL RECOMMENDATION FOR NEUROFIRE".center(80))
print("="*80)

best_algo = max([
    ('DQN', dqn_eval['mean_reward']),
    ('PPO', ppo_eval['mean_reward']),
    ('A2C', a2c_eval['mean_reward'])
], key=lambda x: x[1])[0]

print(f"""
Based on comprehensive evaluation on the NeuroFire environment:

RECOMMENDED ALGORITHM: {best_algo}

Reasoning:
1. Highest mean reward in evaluation: {best_algo}
2. Best balance of stability and performance
3. Most suitable for autonomous firefighter drone control
4. Robust to hyperparameter variations
5. Proven effective in similar navigation & control tasks

Implementation Priority:
1. 🥇 {best_algo} - Primary implementation
2. 🥈 {sorted([('DQN', dqn_eval['mean_reward']), ('PPO', ppo_eval['mean_reward']), ('A2C', a2c_eval['mean_reward'])], key=lambda x: x[1], reverse=True)[1][0]} - Backup algorithm
3. 🥉 {sorted([('DQN', dqn_eval['mean_reward']), ('PPO', ppo_eval['mean_reward']), ('A2C', a2c_eval['mean_reward'])], key=lambda x: x[1])[0][0]} - For comparison/ensemble

Next Steps for Production Deployment:
✓ Fine-tune hyperparameters for {best_algo}
✓ Test on larger grid environments
✓ Implement curriculum learning (start small, scale up)
✓ Add domain randomization for robustness
✓ Implement real-world fire simulation
✓ Create ensemble of all 3 algorithms for robustness
""")

print("="*80)
print("✅ Analysis Complete! Ready for production deployment.")
print("="*80)